In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import pandas as pd
import matplotlib.pyplot as plt

spark = SparkSession.builder \
    .appName("HackathonForecast") \
    .master("local[*]") \
    .config("spark.driver.memory", "8g") \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .getOrCreate()

print("SparkSession creada exitosamente!")

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/09/16 13:21:50 WARN Utils: Your hostname, QUIN-DAT-A0012, resolves to a loopback address: 127.0.1.1; using 192.168.1.12 instead (on interface wlp2s0)
25/09/16 13:21:50 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/09/16 13:21:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


SparkSession creada exitosamente!


In [2]:
from pyspark.sql.window import Window
from pyspark.sql.functions import lag, avg, stddev, col, sum, weekofyear, month, floor

# --- 1. CARGAR DATOS LIMPIOS DE LA CAPA SILVER ---
base_path = "/home/quind/GIT/Desafio-Tecnico-Hackathon-Forecast-Big-Data-2025/"
silver_path = f"{base_path}silver/datos_limpios"

print("Cargando datos limpios desde la capa Silver...")
df_final = spark.read.parquet(silver_path)

# --- 2. CREAR TABLAS DE DIMENSIONES (DESCRIPTIVAS) ---
# Seleccionamos las características únicas de cada producto
features_produto = df_final.select("produto", "categoria", "label", "subcategoria", "marca").distinct()

# Seleccionamos las características únicas de cada PDV
# CAMBIO: Se ha eliminado "zipcode" de la selección de características
features_pdv = df_final.select("pdv", "premise", "categoria_pdv").distinct()

# --- 3. AGREGACIÓN SEMANAL ---
print("Agregando transacciones a nivel semanal...")
df_semanal = df_final.groupBy("pdv", "produto", weekofyear("transaction_date").alias("semana")) \
    .agg(sum("quantity").alias("cantidad_total_semanal"))

# --- 4. ENRIQUECER DATOS SEMANALES CON LAS DIMENSIONES ---
print("Reincorporando las features descriptivas...")
df_enriquecido = df_semanal.join(features_produto, "produto", "left") \
                           .join(features_pdv, "pdv", "left")

# --- 5. CREACIÓN DE FEATURES DE LAG Y VENTANA MÓVIL ---
print("Creando features de Lag y Ventana Móvil...")
# Se recomienda reparticionar antes de la ventana para optimizar el rendimiento
windowSpec = Window.partitionBy("pdv", "produto").orderBy("semana")

df_con_features = df_enriquecido \
    .withColumn("lag_1", lag("cantidad_total_semanal", 1, 0).over(windowSpec)) \
    .withColumn("lag_2", lag("cantidad_total_semanal", 2, 0).over(windowSpec)) \
    .withColumn("lag_4", lag("cantidad_total_semanal", 4, 0).over(windowSpec)) \
    .withColumn("media_movil_4_semanas", avg("cantidad_total_semanal").over(windowSpec.rowsBetween(-3, 0))) \
    .withColumn("stddev_movil_4_semanas", stddev("cantidad_total_semanal").over(windowSpec.rowsBetween(-3, 0)))

# --- 6. CREACIÓN DE FEATURES DE CALENDARIO ---
print("Creando features de calendario...")
df_con_features = df_con_features.withColumn("mes", floor((col("semana") - 1) / 4.34) + 1)

df_listo_para_modelo = df_con_features.fillna(0)

print("\n¡Ingeniería de Features completada! (Ahora con todas las columnas)")
print("Muestra del DataFrame final:")
df_listo_para_modelo.show()

print("\nEsquema final del DataFrame:")
df_listo_para_modelo.printSchema()

Cargando datos limpios desde la capa Silver...
Agregando transacciones a nivel semanal...
Reincorporando las features descriptivas...
Creando features de Lag y Ventana Móvil...
Creando features de calendario...

¡Ingeniería de Features completada! (Ahora con todas las columnas)
Muestra del DataFrame final:


+------------------+-------------------+------+----------------------+---------+---------+------------+--------------------+---------+-------------+-----+-----+-----+---------------------+----------------------+---+
|               pdv|            produto|semana|cantidad_total_semanal|categoria|    label|subcategoria|               marca|  premise|categoria_pdv|lag_1|lag_2|lag_4|media_movil_4_semanas|stddev_movil_4_semanas|mes|
+------------------+-------------------+------+----------------------+---------+---------+------------+--------------------+---------+-------------+-----+-----+-----+---------------------+----------------------+---+
|100190811186115530|1280666905870999728|     2|                   2.0|  PACKAGE|SEM_LABEL|       LAGER|        KIRINICHIBAN|ONPREMISE|        ASIAN|  0.0|  0.0|  0.0|                  2.0|                   0.0|  1|
|100190811186115530|1280666905870999728|    14|                   3.0|  PACKAGE|SEM_LABEL|       LAGER|        KIRINICHIBAN|ONPREMISE|  

In [3]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, FeatureHasher
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col

# --- 1. IDENTIFICAR TIPOS DE FEATURES ---
# La variable objetivo que queremos predecir
TARGET_COL = "cantidad_total_semanal"

# Columnas categóricas que necesitamos codificar
CATEGORICAL_COLS = [
    "pdv", "produto", "categoria", "label", "subcategoria",
    "marca", "premise", "categoria_pdv"
]

# Columnas numéricas que ya están listas para usar
NUMERICAL_COLS = [
    "semana", "lag_1", "lag_2", "lag_4",
    "media_movil_4_semanas", "stddev_movil_4_semanas", "mes"
]

# --- 2. DEFINIR LAS ETAPAS DEL PIPELINE ---

# Etapa 1: StringIndexer - Convierte texto a índices numéricos
indexers = [
    StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep")
    for c in CATEGORICAL_COLS
]
indexed_cols = [f"{c}_idx" for c in CATEGORICAL_COLS]

# Etapa 2: FeatureHasher en lugar de OneHotEncoder
hasher = FeatureHasher(
    inputCols=indexed_cols,
    outputCol="hashed_features",
    numFeatures=1024
)

# Etapa 3: VectorAssembler - Une todas las features en un solo vector
feature_sources = ["hashed_features"] + NUMERICAL_COLS
assembler = VectorAssembler(
    inputCols=feature_sources,
    outputCol="features"
)

# Etapa 4: Modelo - Usaremos Gradient Boosted Trees Regressor
gbt = GBTRegressor(featuresCol="features", labelCol=TARGET_COL)

# Unimos las nuevas etapas en el Pipeline
pipeline = Pipeline(stages=indexers + [hasher, assembler, gbt])

# --- 3. DIVIDIR LOS DATOS (ENTRENAMIENTO Y PRUEBA) ---
print("Dividiendo los datos en entrenamiento y prueba...")
train_data = df_listo_para_modelo.filter(col("semana") <= 40)

# CAMBIO CLAVE: Se han añadido paréntesis a cada condición del filtro
test_data = df_listo_para_modelo.filter( (col("semana") > 40) & (col("semana") < 50) )

print(f"Filas de entrenamiento: {train_data.count()}")
print(f"Filas de prueba: {test_data.count()}")

# --- 4. ENTRENAR EL MODELO ---
print("\nEntrenando el pipeline del modelo ML...")
model = pipeline.fit(train_data)
print("¡Entrenamiento completado!")

# --- 5. REALIZAR PREDICCIONES Y EVALUAR ---
print("\nRealizando predicciones en el conjunto de prueba...")
predictions = model.transform(test_data)

# Mostrar algunas predicciones
predictions.select("pdv", "produto", "semana", TARGET_COL, "prediction").show(10)

# Evaluar el rendimiento del modelo usando RMSE
evaluator = RegressionEvaluator(
    labelCol=TARGET_COL,
    predictionCol="prediction",
    metricName="rmse"
)
rmse = evaluator.evaluate(predictions)
print(f"\nRoot Mean Squared Error (RMSE) en los datos de prueba = {rmse}")

Dividiendo los datos en entrenamiento y prueba...


Filas de entrenamiento: 4296393


Filas de prueba: 1066936

Entrenando el pipeline del modelo ML...


25/09/16 13:24:26 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
25/09/16 13:24:38 WARN DAGScheduler: Broadcasting large task binary with size 1344.4 KiB
25/09/16 13:24:39 WARN DAGScheduler: Broadcasting large task binary with size 1344.2 KiB
25/09/16 13:24:48 WARN DAGScheduler: Broadcasting large task binary with size 1358.3 KiB
25/09/16 13:24:57 WARN DAGScheduler: Broadcasting large task binary with size 1369.5 KiB
25/09/16 13:25:01 WARN MemoryStore: Not enough space to cache rdd_420_7 in memory! (computed 107.7 MiB so far)
25/09/16 13:25:01 WARN MemoryStore: Not enough space to cache rdd_420_13 in memory! (computed 242.4 MiB so far)
25/09/16 13:25:01 WARN MemoryStore: Not enough space to cache rdd_420_10 in memory! (computed 242.4 MiB so far)
25/09/16 13:25:01 WARN MemoryStore: Not enough space to cache rdd_420_6 in memory! (computed 242.4 MiB so far)
25/09/16 

¡Entrenamiento completado!

Realizando predicciones en el conjunto de prueba...


25/09/16 13:38:27 WARN DAGScheduler: Broadcasting large task binary with size 1149.8 KiB


+-------------------+-------------------+------+----------------------+------------------+
|                pdv|            produto|semana|cantidad_total_semanal|        prediction|
+-------------------+-------------------+------+----------------------+------------------+
| 100190811186115530|1280666905870999728|    43|                   1.0|1.6742139918727847|
| 100190811186115530|4767947340064453366|    43|                   2.0| 3.295845629122906|
| 100190811186115530| 801379983953984295|    43|                   2.0|1.3949389834616837|
|1004779246734143594|1009179103632945474|    44|                  12.0| 7.898766799747307|
|1004779246734143594|1029370090212151375|    41|                   1.0|1.4017142091502104|
|1004779246734143594|1029370090212151375|    43|                   2.0|1.7370253666905797|
|1004779246734143594|1029370090212151375|    45|                   3.0|1.8993479907889625|
|1004779246734143594|1029370090212151375|    48|                   3.0|2.1719220896723375|

25/09/16 13:38:40 WARN DAGScheduler: Broadcasting large task binary with size 1345.5 KiB



Root Mean Squared Error (RMSE) en los datos de prueba = 8.125224331564253


25/09/16 13:38:47 WARN DAGScheduler: Broadcasting large task binary with size 1346.6 KiB


In [4]:
# Calcula el promedio y la desviación estándar de la cantidad de ventas
df_listo_para_modelo.select(
    avg("cantidad_total_semanal"),
    stddev("cantidad_total_semanal")
).show()

+---------------------------+------------------------------+
|avg(cantidad_total_semanal)|stddev(cantidad_total_semanal)|
+---------------------------+------------------------------+
|          4.141928978442226|             9.922637981044964|
+---------------------------+------------------------------+



In [5]:
from pyspark.sql.functions import sum as spark_sum, abs as spark_abs

# 'predictions' es tu DataFrame con la columna "prediction" y TARGET_COL

# Asegúrate de que las predicciones no sean negativas
predictions = predictions.withColumn("prediction", when(col("prediction") < 0, 0).otherwise(col("prediction")))

# Calcula el WMAPE
wmape_df = predictions.agg(
    (spark_sum(spark_abs(col(TARGET_COL) - col("prediction"))) / spark_sum(col(TARGET_COL))).alias("wmape")
)

wmape = wmape_df.collect()[0]["wmape"]
print(f"WMAPE en los datos de prueba = {wmape * 100:.2f}%")

25/09/16 14:26:28 WARN DAGScheduler: Broadcasting large task binary with size 1152.6 KiB


WMAPE en los datos de prueba = 30.23%


In [10]:
from pyspark.ml.feature import StringIndexer, VectorAssembler, FeatureHasher
from pyspark.ml.regression import GBTRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.functions import col, when, sum as spark_sum, abs as spark_abs
# CAMBIO: Importar las herramientas para el ajuste de hiperparámetros
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# --- 1. IDENTIFICAR TIPOS DE FEATURES ---
TARGET_COL = "cantidad_total_semanal"
CATEGORICAL_COLS = [
    "pdv", "produto", "categoria", "label", "subcategoria",
    "marca", "premise", "categoria_pdv"
]
NUMERICAL_COLS = [
    "semana", "lag_1", "lag_2", "lag_4",
    "media_movil_4_semanas", "stddev_movil_4_semanas", "mes"
]

# --- 2. DEFINIR LAS ETAPAS DEL PIPELINE ---
# (Esta parte es idéntica a tu código anterior)
indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_idx", handleInvalid="keep") for c in CATEGORICAL_COLS]
indexed_cols = [f"{c}_idx" for c in CATEGORICAL_COLS]
hasher = FeatureHasher(inputCols=indexed_cols, outputCol="hashed_features", numFeatures=1024)
feature_sources = ["hashed_features"] + NUMERICAL_COLS
assembler = VectorAssembler(inputCols=feature_sources, outputCol="features")
gbt = GBTRegressor(featuresCol="features", labelCol=TARGET_COL)
pipeline = Pipeline(stages=indexers + [hasher, assembler, gbt])

# --- 3. DIVIDIR LOS DATOS (ENTRENAMIENTO Y PRUEBA) ---
print("Dividiendo los datos en entrenamiento y prueba...")
train_data = df_listo_para_modelo.filter(col("semana") <= 40)
test_data = df_listo_para_modelo.filter( (col("semana") > 40) & (col("semana") < 50) )

# --- 4. AJUSTE DE HIPERPARÁMETROS ---
print("\nIniciando el ajuste de hiperparámetros...")

# Paso 1: Definir la cuadrícula de parámetros que queremos probar.
# GBTRegressor tiene muchos parámetros, pero estos son de los más importantes.
# CÓDIGO CORRECTO
# Paso 1: Definir la cuadrícula de parámetros que queremos probar.
paramGrid = ParamGridBuilder() \
    .addGrid(gbt.maxDepth, [3, 5]) \
    .addGrid(gbt.maxBins, [32, 64]) \
    .addGrid(gbt.stepSize, [0.1, 0.05]) \
    .build()

# Paso 2: Definir el evaluador que medirá el rendimiento. Usaremos RMSE.
evaluator = RegressionEvaluator(labelCol=TARGET_COL, predictionCol="prediction", metricName="rmse")

# Paso 3: Configurar el CrossValidator.
# Le pasamos el pipeline completo (estimador), la cuadrícula de parámetros y el evaluador.
# numFolds=3 es un buen punto de partida: divide los datos en 3 partes, entrena en 2 y prueba en 1, rotando.
crossval = CrossValidator(estimator=pipeline,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Paso 4: Crear una muestra de los datos para que el ajuste sea rápido.
# Usar el 10% (fraction=0.1) de los datos es una buena estrategia para empezar.
print("Creando una muestra de los datos de entrenamiento para el ajuste...")
train_sample = train_data.sample(withReplacement=False, fraction=0.1, seed=42)
train_sample.cache() # Guardar la muestra en memoria para acelerar el proceso

print(f"Filas en la muestra de entrenamiento: {train_sample.count()}")

# Paso 5: Ejecutar el CrossValidator. Este es el paso que entrena múltiples modelos.
print("Ejecutando CrossValidator... (Esto puede tardar un poco)")
cvModel = crossval.fit(train_sample)
print("¡Ajuste de hiperparámetros completado!")

train_sample.unpersist() # Liberar la muestra de la memoria

# --- 5. REALIZAR PREDICCIONES Y EVALUAR CON EL MEJOR MODELO ---
# `cvModel` es el pipeline que contiene la mejor combinación de hiperparámetros encontrada.
print("\nRealizando predicciones en el conjunto de prueba con el mejor modelo...")
predictions = cvModel.transform(test_data)

# Asegurarse de que las predicciones no sean negativas
predictions = predictions.withColumn("prediction", when(col("prediction") < 0, 0).otherwise(col("prediction")))

# Mostrar algunas predicciones
predictions.select("pdv", "produto", "semana", TARGET_COL, "prediction").show(10)

# Evaluar el rendimiento del mejor modelo
rmse = evaluator.evaluate(predictions)
print(f"\nRoot Mean Squared Error (RMSE) en los datos de prueba (Post-Tuning) = {rmse}")

# Calcular la métrica oficial del concurso: WMAPE
wmape_df = predictions.agg(
    (spark_sum(spark_abs(col(TARGET_COL) - col("prediction"))) / spark_sum(col(TARGET_COL))).alias("wmape")
)
wmape = wmape_df.collect()[0]["wmape"]
print(f"WMAPE en los datos de prueba = {wmape * 100:.2f}%")

Dividiendo los datos en entrenamiento y prueba...

Iniciando el ajuste de hiperparámetros...
Creando una muestra de los datos de entrenamiento para el ajuste...


Filas en la muestra de entrenamiento: 429754
Ejecutando CrossValidator... (Esto puede tardar un poco)


25/09/16 14:33:38 WARN DAGScheduler: Broadcasting large task binary with size 1023.5 KiB
25/09/16 14:33:38 WARN DAGScheduler: Broadcasting large task binary with size 1023.3 KiB
25/09/16 14:33:39 WARN DAGScheduler: Broadcasting large task binary with size 1037.3 KiB
25/09/16 14:33:41 WARN DAGScheduler: Broadcasting large task binary with size 1048.6 KiB
25/09/16 14:33:45 WARN DAGScheduler: Broadcasting large task binary with size 1049.4 KiB
25/09/16 14:33:46 WARN DAGScheduler: Broadcasting large task binary with size 1050.4 KiB
25/09/16 14:33:48 WARN DAGScheduler: Broadcasting large task binary with size 1056.6 KiB
25/09/16 14:33:50 WARN DAGScheduler: Broadcasting large task binary with size 1057.1 KiB
25/09/16 14:33:52 WARN DAGScheduler: Broadcasting large task binary with size 1058.1 KiB
25/09/16 14:33:53 WARN DAGScheduler: Broadcasting large task binary with size 1060.2 KiB
25/09/16 14:33:55 WARN DAGScheduler: Broadcasting large task binary with size 1060.7 KiB
25/09/16 14:33:57 WAR

¡Ajuste de hiperparámetros completado!

Realizando predicciones en el conjunto de prueba con el mejor modelo...


+-------------------+-------------------+------+----------------------+------------------+
|                pdv|            produto|semana|cantidad_total_semanal|        prediction|
+-------------------+-------------------+------+----------------------+------------------+
| 100190811186115530|1280666905870999728|    43|                   1.0| 1.839286576511099|
| 100190811186115530|4767947340064453366|    43|                   2.0| 4.415160483753613|
| 100190811186115530| 801379983953984295|    43|                   2.0|1.5236180813407905|
|1004779246734143594|1009179103632945474|    44|                  12.0|10.144384569611686|
|1004779246734143594|1029370090212151375|    41|                   1.0|1.5236180813407905|
|1004779246734143594|1029370090212151375|    43|                   2.0|1.5236180813407905|
|1004779246734143594|1029370090212151375|    45|                   3.0|1.7462730394382395|
|1004779246734143594|1029370090212151375|    48|                   3.0|1.9678829525487658|

25/09/16 15:50:45 WARN DAGScheduler: Broadcasting large task binary with size 1035.5 KiB
25/09/16 15:50:51 WARN DAGScheduler: Broadcasting large task binary with size 1036.6 KiB



Root Mean Squared Error (RMSE) en los datos de prueba (Post-Tuning) = 8.064800812717781


WMAPE en los datos de prueba = 32.01%


In [11]:
# El 'cvModel' ya contiene el mejor pipeline encontrado durante el CrossValidation
# No es necesario re-entrenar desde cero, cvModel.bestModel ya es el pipeline listo.
best_pipeline_model = cvModel.bestModel

# Opcional pero recomendado: Si quieres estar 100% seguro y tienes tiempo/recursos,
# puedes entrenar este pipeline con el dataset completo de entrenamiento.
# final_model = best_pipeline_model.fit(train_data)
# Para este ejemplo, usaremos el 'best_pipeline_model' que ya está entrenado en la muestra,
# que suele ser suficiente y mucho más rápido.

In [12]:
from pyspark.sql.functions import lit, sequence, explode

# Obtener todas las combinaciones únicas de pdv y produto del dataset completo
pdv_produto_unicos = df_listo_para_modelo.select("pdv", "produto").distinct()

# Crear un DataFrame con las semanas 1 a 5
semanas_enero = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["semana"])

# Hacer un cross join para tener cada pdv/produto para cada semana de enero
df_enero_2023 = pdv_produto_unicos.crossJoin(semanas_enero)

In [13]:
# Tomar los datos de las últimas semanas de 2022 para calcular los lags
df_ultimas_semanas = df_listo_para_modelo.filter(col("semana") >= 49) # Semanas 49, 50, 51, 52

# Crear las features para la semana 1 de 2023 (que sería la semana 53 si el año continuara)
# Por simplicidad, asumiremos que las ventas de las últimas 4 semanas de 2022 son una buena base
# para predecir las primeras de 2023.
last_features = df_ultimas_semanas.groupBy("pdv", "produto") \
    .agg(
        # El lag_1 para la semana 1 de 2023 es la venta de la semana 52 de 2022
        # Esta es una simplificación, un modelo más complejo haría esto iterativamente.
        avg("cantidad_total_semanal").alias("media_movil_4_semanas"),
        stddev("cantidad_total_semanal").alias("stddev_movil_4_semanas"),
        # Usaremos el promedio como proxy para los lags
        avg("cantidad_total_semanal").alias("lag_1"),
        avg("cantidad_total_semanal").alias("lag_2"),
        avg("cantidad_total_semanal").alias("lag_4")
    ).fillna(0)

# Unir estas características al DataFrame de enero
df_enero_con_features = df_enero_2023.join(last_features, ["pdv", "produto"], "left") \
                                     .withColumn("mes", lit(1)) # Enero es el mes 1

# Re-unir las características descriptivas (marca, categoría, etc.)
features_produto = df_listo_para_modelo.select("produto", "categoria", "label", "subcategoria", "marca").distinct()
features_pdv = df_listo_para_modelo.select("pdv", "premise", "categoria_pdv").distinct()

df_enero_final = df_enero_con_features.join(features_produto, "produto", "left") \
                                      .join(features_pdv, "pdv", "left") \
                                      .fillna(0)

In [16]:
# --- 5. ENTRENAR EL MODELO FINAL CON LOS MEJORES PARÁMETROS ---
print("\nPreparando el entrenamiento final con los mejores parámetros...")

# Paso 1: Extraer el mapa de parámetros del mejor modelo que encontró el CrossValidator.
# La última etapa (-1) de nuestro pipeline es el GBTRegressor.
best_gbt_model_params = cvModel.bestModel.stages[-1].extractParamMap()

# Paso 2: Tomamos nuestro pipeline original (que es un estimador, no un modelo entrenado)
# y le aplicamos los parámetros que acabamos de extraer a su etapa de GBTRegressor.
# El método .copy() es la forma segura de hacer esto.
pipeline.getStages()[-1].copy(extra=best_gbt_model_params)

# Paso 3: Ahora que el pipeline original está configurado con los mejores parámetros,
# lo entrenamos con TODOS los datos de entrenamiento.
print(f"Entrenando el modelo final en las {train_data.count()} filas completas de entrenamiento...")
final_model = pipeline.fit(train_data)

print("¡Modelo final entrenado y listo para las predicciones!")

# A partir de aquí, tu código para realizar predicciones funcionará perfectamente
# con el objeto 'final_model'.
# Ejemplo:
# predicciones_enero = final_model.transform(df_enero_final)


Preparando el entrenamiento final con los mejores parámetros...


Entrenando el modelo final en las 4296393 filas completas de entrenamiento...


25/09/16 16:13:31 WARN DAGScheduler: Broadcasting large task binary with size 1344.4 KiB
25/09/16 16:13:32 WARN DAGScheduler: Broadcasting large task binary with size 1344.2 KiB
25/09/16 16:13:43 WARN DAGScheduler: Broadcasting large task binary with size 1358.3 KiB
25/09/16 16:13:53 WARN DAGScheduler: Broadcasting large task binary with size 1369.5 KiB
25/09/16 16:13:58 WARN MemoryStore: Not enough space to cache rdd_14385_8 in memory! (computed 242.4 MiB so far)
25/09/16 16:13:58 WARN MemoryStore: Not enough space to cache rdd_14385_10 in memory! (computed 242.4 MiB so far)
25/09/16 16:13:58 WARN BlockManager: Persisting block rdd_14385_8 to disk instead.
25/09/16 16:13:58 WARN BlockManager: Persisting block rdd_14385_10 to disk instead.
25/09/16 16:13:58 WARN MemoryStore: Not enough space to cache rdd_14385_14 in memory! (computed 242.4 MiB so far)
25/09/16 16:13:58 WARN BlockManager: Persisting block rdd_14385_14 to disk instead.
25/09/16 16:13:59 WARN MemoryStore: Not enough space

[11171,187s][warning][gc,alloc] Executor task launch worker for task 8.0 in stage 12258.0 (TID 867595): Retried waiting for GCLocker too often allocating 86828 words


25/09/16 16:27:59 WARN DAGScheduler: Broadcasting large task binary with size 1507.7 KiB
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_11 in memory! (computed 107.7 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_15 in memory! (computed 161.5 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_6 in memory! (computed 161.5 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_1 in memory! (computed 161.5 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_3 in memory! (computed 30.5 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_9 in memory! (computed 161.5 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_10 in memory! (computed 161.5 MiB so far)
25/09/16 16:28:00 WARN MemoryStore: Not enough space to cache rdd_14385_2 in memory! (computed 161.5 MiB so far)
25/09

¡Modelo final entrenado y listo para las predicciones!


In [22]:
from pyspark.sql.functions import lit, col, when, round, avg, stddev, countDistinct

# --- RE-ENTRENAR Y PREPARAR SUBMISIÓN ---
# (Asumimos que ya tienes tu 'final_model' entrenado con los mejores hiperparámetros y todos los datos)

# --- PASO CLAVE: Crear el DataFrame para Enero 2023 solo con productos FRECUENTES ---
print("Filtrando combinaciones de PDV/Produto con actividad frecuente y reciente...")

# 1. Filtramos por las últimas 10 semanas del año (semanas 43-52) para tener una buena ventana.
df_reciente = df_listo_para_modelo.filter(col("semana") > 42)

# 2. Agrupamos y contamos en cuántas semanas distintas se vendió cada producto en ese periodo.
pdv_produto_frecuentes = df_reciente.groupBy("pdv", "produto") \
                                    .agg(countDistinct("semana").alias("num_semanas_vendidas"))

# 3. CAMBIO CRÍTICO: Nos quedamos solo con los que se vendieron en 2 o más semanas.
pdv_produto_recientes = pdv_produto_frecuentes.filter(col("num_semanas_vendidas") >= 2) \
                                              .select("pdv", "produto")
                                            
# Cachear este DataFrame puede acelerar el siguiente paso
pdv_produto_recientes.cache()
print(f"Se encontraron {pdv_produto_recientes.count()} combinaciones activas y frecuentes.")

# Crear un DataFrame con las semanas 1 a 5 de Enero
semanas_enero = spark.createDataFrame([(1,), (2,), (3,), (4,), (5,)], ["semana"])

# Hacer un cross join para tener cada pdv/produto ACTIVO para cada semana de enero
df_enero_2023 = pdv_produto_recientes.crossJoin(semanas_enero)

# --- Ingeniería de Características para el Futuro ---
print("Generando características para las predicciones de Enero...")

# Tomar los datos de las últimas semanas de 2022 para calcular los lags y medias móviles
df_ultimas_semanas = df_listo_para_modelo.filter(col("semana") >= 49)

last_features = df_ultimas_semanas.groupBy("pdv", "produto") \
    .agg(
        avg("cantidad_total_semanal").alias("media_movil_4_semanas"),
        stddev("cantidad_total_semanal").alias("stddev_movil_4_semanas"),
        avg("cantidad_total_semanal").alias("lag_1"),
        avg("cantidad_total_semanal").alias("lag_2"),
        avg("cantidad_total_semanal").alias("lag_4")
    ).fillna(0)

# Unir estas características al DataFrame de enero
df_enero_con_features = df_enero_2023.join(last_features, ["pdv", "produto"], "left") \
                                     .withColumn("mes", lit(1))

# Re-unir las características descriptivas (marca, categoría, etc.)
features_produto = df_listo_para_modelo.select("produto", "categoria", "label", "subcategoria", "marca").distinct()
features_pdv = df_listo_para_modelo.select("pdv", "premise", "categoria_pdv").distinct()

df_enero_final = df_enero_con_features.join(features_produto, "produto", "left") \
                                      .join(features_pdv, "pdv", "left") \
                                      .fillna(0)

# Liberar el DataFrame cacheado
pdv_produto_recientes.unpersist()

# --- Realizar las Predicciones Finales y Guardar ---
print("Realizando predicciones para Enero 2023...")
predicciones_enero = final_model.transform(df_enero_final)

# Formatear el resultado según las reglas del concurso
df_submission = predicciones_enero.select(
    col("semana"),
    col("pdv"),
    col("produto"),
    when(col("prediction") < 0, 0).otherwise(round(col("prediction"))).cast("integer").alias("quantidade")
)

# Filtrar también las predicciones que sean 0 para reducir aún más el tamaño
df_submission = df_submission.filter(col("quantidade") > 0)

print(f"Número final de filas a guardar: {df_submission.count()}")
df_submission.show(10)

# Guardar el archivo final en formato Parquet
submission_path = "/home/quind/GIT/Desafio-Tecnico-Hackathon-Forecast-Big-Data-2025/submission_parquet"
print(f"Guardando archivo de submisión en formato Parquet en: {submission_path}")

df_submission.repartition(1).write.mode("overwrite").parquet(submission_path)

print("¡Archivo de submisión en formato Parquet generado exitosamente!")

Filtrando combinaciones de PDV/Produto con actividad frecuente y reciente...


Se encontraron 272186 combinaciones activas y frecuentes.
Generando características para las predicciones de Enero...
Realizando predicciones para Enero 2023...


25/09/16 16:54:42 WARN DAGScheduler: Broadcasting large task binary with size 1078.0 KiB


Número final de filas a guardar: 1360930


25/09/16 16:55:27 WARN DAGScheduler: Broadcasting large task binary with size 1484.9 KiB


+------+-------------------+-------------------+----------+
|semana|                pdv|            produto|quantidade|
+------+-------------------+-------------------+----------+
|     1|1004779246734143594|1029370090212151375|         2|
|     2|1004779246734143594|1029370090212151375|         2|
|     3|1004779246734143594|1029370090212151375|         2|
|     4|1004779246734143594|1029370090212151375|         2|
|     5|1004779246734143594|1029370090212151375|         2|
|     1|1004779246734143594|1657665165780983454|        11|
|     2|1004779246734143594|1657665165780983454|        11|
|     3|1004779246734143594|1657665165780983454|        11|
|     4|1004779246734143594|1657665165780983454|        11|
|     5|1004779246734143594|1657665165780983454|        11|
+------+-------------------+-------------------+----------+
only showing top 10 rows
Guardando archivo de submisión en formato Parquet en: /home/quind/GIT/Desafio-Tecnico-Hackathon-Forecast-Big-Data-2025/submission_parqu

25/09/16 16:56:12 WARN DAGScheduler: Broadcasting large task binary with size 1483.7 KiB


¡Archivo de submisión en formato Parquet generado exitosamente!


In [19]:
# --- Realizar las Predicciones Finales y Guardar ---
print("Realizando predicciones para Enero 2023...")
predicciones_enero = final_model.transform(df_enero_final)

# Formatear el resultado según las reglas del concurso
df_submission = predicciones_enero.select(
    col("semana"),
    col("pdv"),
    col("produto"),
    when(col("prediction") < 0, 0).otherwise(round(col("prediction"))).cast("integer").alias("quantidade")
)

# --- CAMBIO CLAVE: Filtrar las predicciones que sean 0 ---
# Descomentamos la siguiente línea para eliminar las filas con predicciones de cero ventas.
# Esto reducirá drásticamente el número de filas para cumplir con el límite.
df_submission = df_submission.filter(col("quantidade") > 0)

print(f"Número final de filas a guardar (solo con ventas > 0): {df_submission.count()}")
df_submission.show(10)

# Guardar el archivo final en formato Parquet
submission_path = "/home/quind/GIT/Desafio-Tecnico-Hackathon-Forecast-Big-Data-2025/submission_parquet"
print(f"Guardando archivo de submisión en formato Parquet en: {submission_path}")

df_submission.repartition(1).write.mode("overwrite").parquet(submission_path)

print("¡Archivo de submisión generado exitosamente!")

Realizando predicciones para Enero 2023...


25/09/16 16:44:27 WARN DAGScheduler: Broadcasting large task binary with size 1472.9 KiB


Número final de filas a guardar (solo con ventas > 0): 3457575


25/09/16 16:45:29 WARN DAGScheduler: Broadcasting large task binary with size 1483.6 KiB


+------+------------------+-------------------+----------+
|semana|               pdv|            produto|quantidade|
+------+------------------+-------------------+----------+
|     1|100190811186115530|4767947340064453366|         1|
|     2|100190811186115530|4767947340064453366|         1|
|     3|100190811186115530|4767947340064453366|         1|
|     4|100190811186115530|4767947340064453366|         1|
|     5|100190811186115530|4767947340064453366|         1|
|     1|100190811186115530| 801379983953984295|         1|
|     2|100190811186115530| 801379983953984295|         1|
|     3|100190811186115530| 801379983953984295|         1|
|     4|100190811186115530| 801379983953984295|         1|
|     5|100190811186115530| 801379983953984295|         1|
+------+------------------+-------------------+----------+
only showing top 10 rows
Guardando archivo de submisión en formato Parquet en: /home/quind/GIT/Desafio-Tecnico-Hackathon-Forecast-Big-Data-2025/submission_parquet


25/09/16 16:46:22 WARN DAGScheduler: Broadcasting large task binary with size 1477.9 KiB


¡Archivo de submisión generado exitosamente!
